In [1]:
import av
import numpy as np
import pandas as pd

def sync_video_and_steering(example_segment):
    """
    Sync video frames with steering angle data
    """
    # Load steering data
    steering_times = np.load(example_segment + 'processed_log/CAN/steering_angle/t')
    steering_angles = np.load(example_segment + 'processed_log/CAN/steering_angle/value')
    
    # Load frame times
    frame_times = np.load(example_segment + 'global_pose/frame_times')
    
    print(f"Number of frames: {len(frame_times)}")
    print(f"Number of steering measurements: {len(steering_times)}")
    print(f"\nFrame time range: {frame_times[0]:.2f} to {frame_times[-1]:.2f}")
    print(f"Steering time range: {steering_times[0]:.2f} to {steering_times[-1]:.2f}")
    
    # Create a list to store synchronized data
    synced_data = []
    
    # For each frame time, find the closest steering angle measurement
    for frame_idx, frame_time in enumerate(frame_times):
        # Find the closest steering measurement
        closest_idx = np.argmin(np.abs(steering_times - frame_time))
        
        synced_data.append({
            'frame_idx': frame_idx,
            'frame_time': frame_time,
            'steering_time': steering_times[closest_idx],
            'steering_angle': steering_angles[closest_idx],
            'time_diff': abs(frame_time - steering_times[closest_idx])
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(synced_data)
    
    print("\nSync Statistics:")
    print(f"Average time difference: {df['time_diff'].mean()*1000:.2f} ms")
    print(f"Max time difference: {df['time_diff'].max()*1000:.2f} ms")
    
    # Save synchronized data
    df.to_csv('synced_steering_data.csv', index=False)
    print("\nSynchronized data saved to 'synced_steering_data.csv'")
    
    return df

if __name__ == "__main__":
    example_segment = 'data/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b|2018-07-27--06-03-57/3/'
    df = sync_video_and_steering(example_segment)

Number of frames: 1197
Number of steering measurements: 4973

Frame time range: 9314.09 to 9374.04
Steering time range: 9314.16 to 9374.15

Sync Statistics:
Average time difference: 4.31 ms
Max time difference: 65.34 ms

Synchronized data saved to 'synced_steering_data.csv'
